# Basic Plotting

Use the file: /glade/scratch/hannay/archive/b1850_high_freq/atm/hist/b1850_high_freq.cam.h0.0001-01.nc

## Exercise 1: Make a lat-lon plot of TS

## Exercise 2: Zonal plot of SWCF

## Exercise 3: Plot of zonal T 